# Creating a combined quantification pipeline & batch processing - part 11.5
--------------------

## OVERVIEW
Now that all the function to quantify features of organelle composition, morphology, contacts, and distribution have been created, we will combine them into a wrapper function for easy use.


## OBJECTIVE: ✅ Create a combined quantification pipeline for batch processing all organelles of interest
In this notebook, the logic for quantifying the composition, morphology, contacts, and distribution of organelles (as many as you would like) from single cells. To do this, the quantification functions from previous notebooks at combined into a single function to ***process multiple organelles*** at a time and a framework for ***batch processing*** multiple cells/images is also established here.

## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
import itertools

import parse

import napari

from skimage.measure import (regionprops, _regionprops, regionprops_table)

from napari.utils.notebook_display import nbscreenshot

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_my_props_to_dict, _assert_uint16_labels, get_region_morphology_3D)
from infer_subc.utils.stats_helpers import *

from infer_subc.organelles import * 

from infer_subc.constants import (TEST_IMG_N,
                                    NUC_CH ,
                                    LYSO_CH ,
                                    MITO_CH ,
                                    GOLGI_CH ,
                                    PEROX_CH ,
                                    ER_CH ,
                                    LD_CH ,
                                    RESIDUAL_CH ) 

import time
%load_ext autoreload
%autoreload 2

## Get and load image for quantification
Specifically, this will include the raw image and the outputs from segmentation.

In [2]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"

raw_data_path = data_root_path / "raw"
im_type = ".czi"

raw_file_list = list_image_files(raw_data_path,im_type)
raw_img_name = raw_file_list[test_img_n]

# adding an additional list of image paths for the matching segmentation files
seg_data_path = data_root_path / "out"
seg_file_list = list_image_files(seg_data_path, "tiff")

# changing output directory for this notebook to a new folder called "quant"
out_data_path = data_root_path / "quant"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [3]:
# raw image
raw_img_data, raw_meta_dict = read_czi_image(raw_img_name)

channel_names = raw_meta_dict['name']
img = raw_meta_dict['metadata']['aicsimage']
scale = raw_meta_dict['scale']
channel_axis = raw_meta_dict['channel_axis']

In [4]:
## For each import, change the string to match the suffix on the segmentation files (i.e., the stuff following the "-")

# masks
nuc_seg = import_inferred_organelle("20230426_test_nuc", raw_meta_dict, seg_data_path)
cell_seg = import_inferred_organelle("20230426_test_cell", raw_meta_dict, seg_data_path)
cyto_seg = import_inferred_organelle("20230426_test_cyto", raw_meta_dict, seg_data_path)
# mask_seg = import_inferred_organelle("masks", raw_meta_dict, seg_data_path)

#organelles
lyso_seg = import_inferred_organelle("20230426_test_lyso", raw_meta_dict, seg_data_path)
mito_seg = import_inferred_organelle("20230426_test_mito", raw_meta_dict, seg_data_path)
golgi_seg = import_inferred_organelle("20230426_test_golgi", raw_meta_dict, seg_data_path)
perox_seg = import_inferred_organelle("20230426_test_perox", raw_meta_dict, seg_data_path)
ER_seg = import_inferred_organelle("20230426_test_ER", raw_meta_dict, seg_data_path)
LD_seg = import_inferred_organelle("20230426_test_LD", raw_meta_dict, seg_data_path)


loaded  inferred 3D `20230426_test_nuc`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cell`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cyto`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_lyso`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_mito`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_golgi`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_perox`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_ER`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_LD`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 


# putting it all together

`make_organelle_metrics_tables` prototype

### Testing how to make pairs of organelles with no redundancy

In [ ]:
test_lyso_metrics, rp = get_org_morphology_3D(lyso_seg, raw_img_data[LYSO_CH], cell_seg, scale)
test_lyso_metrics

In [ ]:
# names of organelles we have
organelle_names = ["lyso", "mito","golgi","perox","ER","LD"]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

# create intensities from raw as list
intensities = [raw_img_data[ch] for ch in organelle_channels]

# load organelles as list
organelles = [lyso_seg, mito_seg, golgi_seg, perox_seg, ER_seg, LD_seg]

In [ ]:
# merged_tabs = []
# for pair in test_contact_combos:

a_name = 'lyso' #pair[0]
b_name = 'mito' #pair[1]

i = organelle_names.index(a_name)
j = organelle_names.index(b_name)

a = _assert_uint16_labels(organelles[i]) # org_obj
b = _assert_uint16_labels(organelles[j])

ov = []
b_labs = []
labs = []
# for idx, lab in enumerate(a_stats_tab["label"]):  # a_stats_tab is the data associated to the a organelle collected earlier
for idx, lab in enumerate(test_lyso_metrics["label"]):
    xyz = tuple(rp[idx].coords.T) # change to xyz = tuple(rps[i].coords.T)
    cmp_org = b[xyz]
    
    # total number of overlapping pixels
    overlap = sum(cmp_org > 0)
    # overlap?
    labs_b = cmp_org[cmp_org > 0]
    b_js = np.unique(labs_b).tolist()

    # if overlap > 0:
    labs.append(id) # labs.append(lab)
    ov.append(overlap)
    b_labs.append(b_js)

# add organelle B columns to A_stats_tab
test_lyso_metrics[f"{b_name}_overlap"] = ov
test_lyso_metrics[f"{b_name}_labels"] = b_labs  # might want to make this easier for parsing later

#####  2  ###########
# get cross_stats

cross_tab = get_contact_metrics_3D(a, a_name, b, b_name, cell_seg) 
# shell_cross_tab = get_contact_metrics_3D(a, b, organelle_mask, use_shell_a=True)
            
# cross_tab["organelle_b"]=nmi
# shell_cross_tab["organelle_b"]=nmi
#  Merge cross_tabs and shell_cross_tabs 
# merged_tab = pd.merge(cross_tab,shell_cross_tab, on="label_")
# merged_tab = pd.concat([cross_tab,shell_cross_tab])
# cross_tab.insert(loc=0,column='organelle_b',value=b_name)

cross_tab

In [ ]:
test_contact_combos = list(itertools.combinations(organelle_names, 2))
test_contact_combos

In [ ]:
test_contact_combos = list(itertools.combinations(organelle_names, 2))

contact_tabs = []
for pair in test_contact_combos:
    a_name = pair[0]
    b_name = pair[1]

    i = organelle_names.index(a_name)
    j = organelle_names.index(b_name)

    a = _assert_uint16_labels(organelles[i]) # org_obj
    b = _assert_uint16_labels(organelles[j])

    org_a_tab = org_tabs[i]
    org_a_rp = rps[i]

    ov = []
    b_labs = []
    labs = []
    # for idx, lab in enumerate(a_stats_tab["label"]):  # a_stats_tab is the data associated to the a organelle collected earlier
    for idx, lab in enumerate(org_a_tab["label"]):
        xyz = tuple(org_a_rp[idx].coords.T) # change to xyz = tuple(rps[i].coords.T)
        cmp_org = b[xyz]
        
        # total number of overlapping pixels
        overlap = sum(cmp_org > 0)
        # overlap?
        labs_b = cmp_org[cmp_org > 0]
        b_js = np.unique(labs_b).tolist()

        # if overlap > 0:
        labs.append(id) # labs.append(lab)
        ov.append(overlap)
        b_labs.append(b_js)

    cname = organelle_to_colname[b_name]


    cross_tab = get_contact_metrics_3D(a, a_name, b, b_name, cell_seg) 


    contact_tabs.append(cross_tab)

### Final function

In [5]:
organelle_to_colname = {"nuc":"NU", "lyso": "LY", "mito":"MT", "golgi":"GL", "perox":"PR", "ER":"ER", "LD":"LD", "cell":"CM", "cyto":"CY", "nucleus": "N1","nuclei":"NU",}

def _make_all_metrics_tables(source_file: str,
                             list_obj_names: List[str],
                             list_obj_segs: List[np.ndarray],
                             list_intensity_img: List[np.ndarray],
                             list_region_names: List[str],
                             list_region_segs: List[np.ndarray],
                             mask: str,
                             dist_centering_obj:str, 
                             dist_num_bins: int,
                             dist_center_on: bool=False,
                             dist_keep_center_as_bin: bool=True,
                             dist_zernike_degrees: Union[int, None]=None,
                             scale: Union[tuple,None] = None,
                             include_contact_dist:bool=True):
    """
    Measure the composition, morphology, distribution, and contacts of multiple organelles in a cell

    Parameters:
    ----------
    source_file: str
        file path; this is used for recorder keeping of the file name in the output data tables
    list_obj_names: List[str]
        a list of object names (strings) that will be measured; this should match the order in list_obj_segs
    list_obj_segs: List[np.ndarray]
        a list of 3D (ZYX) segmentation np.ndarrays that will be measured per cell; the order should match the list_obj_names 
    list_intensity_img: List[np.ndarray]
        a list of 3D (ZYX) grayscale np.ndarrays that will be used to measure fluoresence intensity in each region and object
    list_region_names: List[str]
        a list of region names (strings); these should include the mask (entire region being measured - usually the cell) 
        and other sub-mask regions from which we can meausure the objects in (ex - nucleus, neurites, soma, etc.). It should 
        also include the centering object used when created the XY distribution bins.
        The order should match the list_region_segs
    list_region_segs: List[np.ndarray]
        a list of 3D (ZYX) binary np.ndarrays of the region masks; the order should match the list_region_names.
    mask: str
        a str of which region name (contained in the list_region_names list) should be used as the main mask (e.g., cell mask)
    dist_centering_obj:str
        a str of which region name (contained in the list_region_names list) should be used as the centering object in 
        get_XY_distribution()
    dist_num_bins: int
        the number of concentric rings to draw between the centering object and edge of the mask in get_XY_distribution()
    dist_center_on: bool=False,
        for get_XY_distribution:
        True = distribute the bins from the center of the centering object
        False = distribute the bins from the edge of the centering object
    dist_keep_center_as_bin: bool=True
        for get_XY_distribution:
        True = include the centering object area when creating the bins
        False = do not include the centering object area when creating the bins
    dist_zernike_degrees: Union[int, None]=None
        for get_XY_distribution:
        the number of zernike degrees to include for the zernike shape descriptors; if None, the zernike measurements will not 
        be included in the output
    scale: Union[tuple,None] = None
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)
    include_contact_dist:bool=True
        whether to include the distribution of contact sites in get_contact_metrics_3d(); True = include contact distribution

    Returns:
    ----------
    4 Dataframes of measurements of organelle morphology, region morphology, contact morphology, and organelle/contact distributions

    """
    start = time.time()
    count = 0

    # segmentation image for all masking steps below
    mask = list_region_segs[list_region_names.index(mask)]

    ######################
    # measure cell regions
    ######################
    # create np.ndarray of intensity images
    raw_image = np.stack(list_intensity_img)
    
    # container for region data
    region_tabs = []
    for r, r_name in enumerate(list_region_names):
        region = list_region_segs[r]
        region_metrics = get_region_morphology_3D(region_seg=region, 
                                                  region_name=r_name,
                                                  channel_names=list_obj_names,
                                                  intensity_img=raw_image, 
                                                  mask=mask,
                                                  scale=scale)
        region_tabs.append(region_metrics)

    ##############################################################
    # loop through all organelles to collect measurements for each
    ##############################################################
    # containers to collect per organelle information
    org_tabs = []
    dist_tabs = []
    XY_bins = []
    XY_wedges = []

    for j, target in enumerate(list_obj_names):
        # organelle intensity image
        org_img = list_intensity_img[j]

        # organelle segmentation
        if target == 'ER':
            # ensure ER is only one object
            org_obj = (list_obj_segs[j] > 0).astype(np.uint16)
        else:
            org_obj = list_obj_segs[j]

        ##########################################################
        # measure organelle morphology & number of objs contacting
        ##########################################################
        org_metrics = get_org_morphology_3D(segmentation_img=org_obj, 
                                            seg_name=target,
                                            intensity_img=org_img, 
                                            mask=mask,
                                            scale=scale)

        ### org_metrics.insert(loc=0,column='cell',value=1) 
        # ^^^ saving this thought for later when someone might have more than one cell per image.
        # Not sure how they analysis process would fit in our pipelines as they exist now. 
        # Maybe here, iterating though the index of the masks above all of this and using that index as the cell number?

        # TODO: find a better way to quantify the number and area of contacts per organelle
            # I think it can be done during summarizing based on the label and object values in the contact sheet
        # for i, nmi in enumerate(list_obj_names):
        #     if i != j:
        #         if target == 'ER':
        #             b = (list_obj_segs[i] > 0).astype(np.uint16)
        #         else:
        #             b = list_obj_segs[i]
            
        #         ov = []
        #         b_labs = []
        #         labs = []
        #         for idx, lab in enumerate(org_metrics["label"]):
        #             xyz = tuple(rp[idx].coords.T)
        #             cmp_org = b[xyz]
                    
        #             # total area (in voxels or real world units) where these two orgs overlap within the cell
        #             if scale != None:
        #                 overlap = sum(cmp_org > 0)*scale[0]*scale[1]*scale[2]
        #             else:
        #                 # total number of overlapping pixels
        #                 overlap = sum(cmp_org > 0)
        #                 # overlap?
                    
        #             # which b organelles are involved in that overlap
        #             labs_b = cmp_org[cmp_org > 0]
        #             b_js = np.unique(labs_b).tolist()

        #             # if overlap > 0:
        #             labs.append(lab) # labs.append(lab)
        #             ov.append(overlap)
        #             b_labs.append(b_js)
        #         org_metrics[f"{nmi}_overlap"] = ov
        #         org_metrics[f"{nmi}_labels"] = b_labs 

        org_tabs.append(org_metrics)

        ################################
        # measure organelle distribution 
        ################################
        centering = list_region_segs[list_region_names.index(dist_centering_obj)]
        XY_org_distribution, XY_bin_masks, XY_wedge_masks = get_XY_distribution(mask=mask,
                                                                                centering_obj=centering,
                                                                                obj=org_obj,
                                                                                obj_name=target,
                                                                                scale=scale,
                                                                                num_bins=dist_num_bins,
                                                                                center_on=dist_center_on,
                                                                                keep_center_as_bin=dist_keep_center_as_bin,
                                                                                zernike_degrees=dist_zernike_degrees)
        Z_org_distribution = get_Z_distribution(mask=mask, 
                                                obj=org_obj,
                                                obj_name=target,
                                                center_obj=centering,
                                                scale=scale)
        
        org_distribution_metrics = pd.merge(XY_org_distribution, Z_org_distribution,on=["object", "scale"])

        dist_tabs.append(org_distribution_metrics)
        XY_bins.append(XY_bin_masks)
        XY_wedges.append(XY_wedge_masks)

    #######################################
    # collect non-redundant contact metrics 
    #######################################
    # list the non-redundant organelle pairs
    contact_combos = list(itertools.combinations(list_obj_names, 2))

    # container to keep contact data in
    contact_tabs = []

    # loop through each pair and measure contacts
    for pair in contact_combos:
        # pair names
        a_name = pair[0]
        b_name = pair[1]

        # segmentations to measure
        if a_name == 'ER':
            # ensure ER is only one object
            a = (list_obj_segs[list_obj_names.index(a_name)] > 0).astype(np.uint16)
        else:
            a = list_obj_segs[list_obj_names.index(a_name)]
        
        if b_name == 'ER':
            # ensure ER is only one object
            b = (list_obj_segs[list_obj_names.index(b_name)] > 0).astype(np.uint16)
        else:
            b = list_obj_segs[list_obj_names.index(b_name)]
        

        if include_contact_dist == True:
            contact_tab, contact_dist_tab = get_contact_metrics_3D(a, a_name, 
                                                                   b, b_name, 
                                                                   mask, 
                                                                   scale, 
                                                                   include_dist=include_contact_dist,
                                                                   dist_centering_obj=centering,
                                                                   dist_num_bins=dist_num_bins,
                                                                   dist_zernike_degrees=dist_zernike_degrees,
                                                                   dist_center_on=dist_center_on,
                                                                   dist_keep_center_as_bin=dist_keep_center_as_bin)
            dist_tabs.append(contact_dist_tab)
        else:
            contact_tab = get_contact_metrics_3D(a, a_name, 
                                                 b, b_name, 
                                                 mask, 
                                                 scale, 
                                                 include_dist=include_contact_dist)
        contact_tabs.append(contact_tab)


    ###########################################
    # combine all tabs into one table per type:
    ###########################################
    final_org_tab = pd.concat(org_tabs, ignore_index=True)
    final_org_tab.insert(loc=0,column='image_name',value=source_file.stem)

    final_contact_tab = pd.concat(contact_tabs, ignore_index=True)
    final_contact_tab.insert(loc=0,column='image_name',value=source_file.stem)

    combined_dist_tab = pd.concat(dist_tabs, ignore_index=True)
    combined_dist_tab.insert(loc=0,column='image_name',value=source_file.stem)

    final_region_tab = pd.concat(region_tabs, ignore_index=True)
    final_region_tab.insert(loc=0,column='image_name',value=source_file.stem)

    end = time.time()
    print(f"It took {(end-start)/60} minutes to quantify one image.")
    return final_org_tab, final_contact_tab, combined_dist_tab, final_region_tab

In [6]:
# TODO: things to fix - 
# figure out what is causing the convex hull error - I'm guessing major axis
organelle_names = ["lyso", "mito","golgi","perox","ER","LD"]
organelles = [lyso_seg, mito_seg, golgi_seg, perox_seg, ER_seg, LD_seg]
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]
intensities = [raw_img_data[ch] for ch in organelle_channels]
regions = [cell_seg, cyto_seg, nuc_seg]
region_names = ['cell', 'cyto', 'nuc']

test_final_org_tab, test_final_contact_tab, test_combined_dist_tab, test_final_regions_tab = _make_all_metrics_tables(source_file=raw_img_name,
                                                                                                                      list_obj_names=organelle_names,
                                                                                                                      list_obj_segs=organelles,
                                                                                                                      list_intensity_img=intensities,
                                                                                                                      list_region_names=region_names,
                                                                                                                      list_region_segs=regions,
                                                                                                                      mask='cell',
                                                                                                                      dist_centering_obj='nuc',
                                                                                                                      dist_num_bins=5,
                                                                                                                      dist_center_on=False,
                                                                                                                      dist_keep_center_as_bin=True,
                                                                                                                      dist_zernike_degrees=9,
                                                                                                                      scale=scale,
                                                                                                                      include_contact_dist=True)

import warnings
warnings.simplefilter('ignore')

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 251930411  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  6  Error-roundoff 8.1e-15  _one-merge 5.6e-14
  _near-inside 2.8e-13  Visible-distance 1.6e-14  U-max-coplanar 1.6e-14
  Width-outside 3.2e-14  _wide-facet 9.7e-14  _maxoutside 6.4e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:630: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Fai

It took 3.010769251982371 minutes to quantify one image.


In [ ]:
test_final_regions_tab

In [ ]:
test_final_org_tab

In [ ]:
test_final_contact_tab

In [ ]:
test_combined_dist_tab

In [7]:
from infer_subc.utils.stats_helpers import make_all_metrics_tables

official_org, official_contacts, official_dist, official_regions = make_all_metrics_tables(source_file=raw_img_name,
                                                                                           list_obj_names=organelle_names,
                                                                                           list_obj_segs=organelles,
                                                                                           list_intensity_img=intensities,
                                                                                           list_region_names=region_names,
                                                                                           list_region_segs=regions,
                                                                                           mask='cell',
                                                                                           dist_centering_obj='nuc',
                                                                                           dist_num_bins=5,
                                                                                           dist_center_on=False,
                                                                                           dist_keep_center_as_bin=True,
                                                                                           dist_zernike_degrees=9,
                                                                                           scale=scale,
                                                                                           include_contact_dist=True)

test_final_org_tab.equals(official_org), test_final_contact_tab.equals(official_contacts), test_combined_dist_tab.equals(official_dist), test_final_regions_tab.equals(official_regions)

It took 3.152917432785034 minutes to quantify one image.


(True, True, False, True)

In [8]:
test_combined_dist_tab.compare(official_dist)

zernike_obj_mag   
                                                 self   
8   [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...  \
15  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
19  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   

                                                       
                                                other  
8   [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...  
15  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...  
19  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...